#### 사용자 가중치 기반 추천 관광지 리스트

1. 가상의 사용자(별점,리뷰,별점가중치 우선순위 별 5000명씩) 15000명 생성
2. 사용자 클러스터링(가중치)
3. 관광지 클러스터링(가중치 해당 변수 score)
4. 관광지 클러스터링 결과 보고 가중치 우선순위에 맞게 사용자 관광지 클러스터 선택
- 클러스터 0: 별점x, 리뷰 낮음, 가격 중간 => 리뷰 수는 적지만 가격이 좀 있음 ->가격 1순위인 사용자
- 클러스터 1: 별점,리뷰,가격 높음 -> 가격,리뷰,별점 1순위인 사용자
- 클러스터 2: 별점x, 리뷰 중간, 가격x => 리뷰 1순위인 사용자
- 가격 1순위 사용자 : 클러스터 0,1 중 랜덤 1개 선택 -> 선택 클러스터 중 랜덤 관광지 1개 선택
- 리뷰 1순위 사용자 : 클러스터 1,2 중 랜덤 1개 선택 -> 선택 클러스터 중 랜덤 관광지 1개 선택
- 별점 1순위 사용자 : 클러스터 2 -> 선택 클러스터 중 랜덤 관광지 1개 선택
5. 새로운 사용자 가중치 입력
6. 유사한 사용자 식별 -> 가중치 비슷한 사용자(코사인유사도, 유클리안 거리 : 값 동일하게 나와 코사인 유사도 사용)
7. 관광지 추천 -> 유사한 사용자가 선택한 관광지 확인 후, 해당 관광지와 연관된(유사한) 다른 관광지 추천
8. top 40 관광지 리스트 추출

In [6]:
import pandas as pd
import numpy as np
import random
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

user_data = pd.read_csv('./user_data_select.csv')
tourist_data = pd.read_csv('./sight_score.csv')

new_user = [0.5,0.5,0.5]

user_data_numeric = user_data[['price_weight', 'review_weight', 'rating_weight']]


# 코사인 유사도
similarities = cosine_similarity(user_data_numeric, [new_user])

# 유사도가 가장 높은 사용자
most_similar_user_index = similarities.argmax()
most_similar_user = user_data.iloc[most_similar_user_index]
print("가장 유사한 사용자 데이터:", most_similar_user)

# 유사한 사용자가 선택한 관광지 데이터
selected_tourist_data = tourist_data[tourist_data['modelid'] == most_similar_user['select_p_id']]
# 필요한 컬럼 선택
selected_features = ['price_score', 'review_score', 'rating_score', 'modelid']
t_tourist_data = tourist_data[selected_features]
t_selected_tourist_data = selected_tourist_data[selected_features]

# 유사한 사용자가 선택한 관광지와 유사한 관광지
recommendations = []
for tourist_id in t_selected_tourist_data['modelid']:
    similarity_scores = cosine_similarity(t_tourist_data.loc[t_tourist_data['modelid'] == tourist_id].drop(['modelid'], axis=1), t_tourist_data.drop(['modelid'], axis=1))
    similar_tourists = t_tourist_data['modelid'].iloc[similarity_scores[0].argsort()[-40:-1]]
    recommendations.extend(similar_tourists)
    
final_recommendations = list(set(recommendations))

recommended_tourist_data = tourist_data[tourist_data['modelid'].isin(final_recommendations)]
recommended_tourist_data # 유사도 순

가장 유사한 사용자 데이터: price_weight                0.246443
review_weight               0.500006
rating_weight               0.253551
model_user               8415.000000
user_cluster                1.000000
selected_spot_cluster       2.000000
select_p_id               199.000000
Name: 8414, dtype: float64


,modelid,rating_score,review_score,price_score,name,type,address1,address2,rating,themenum,latitude,longitude,themenum.1
242,243,0.0,0.597953,0.0,어승생악,오름,없음,제주 제주시 해안동 산220-12,0.0,21,33.383654,126.509418,21
244,245,0.0,0.579666,0.0,동백동산,"숲,숲길",없음,제주 제주시 조천읍 선흘리 산12,0.0,21,33.516634,126.717031,21
247,248,0.0,0.590996,0.0,알작지,"해수욕장,해변",제주 제주시 테우해안로 60,내도동 475,0.0,21,33.496179,126.441240,21
253,254,0.0,0.568246,0.0,어영공원,근린공원,없음,제주 제주시 용담삼동 2396-16,0.0,21,33.517320,126.486306,21
255,256,0.0,0.581729,0.0,제주올레길6코스,도보코스,없음,제주 서귀포시 보목동 1377-4,0.0,21,33.237150,126.596001,21
257,258,0.0,0.562126,0.0,박수기정,지역명소,없음,제주 서귀포시 안덕면 감산리 1008,0.0,21,33.236629,126.359219,21
259,260,0.0,0.614789,0.0,자구리공원,"도시,테마공원",없음,제주 서귀포시 서귀동 70-1,0.0,21,33.243463,126.569095,21
260,261,0.0,0.566367,0.0,수월봉 지질트레일,도보여행,없음,제주 제주시 한경면 고산리 3760,0.0,21,33.294765,126.163419,21
261,262,0.0,0.638559,0.0,성산항,항구,제주 서귀포시 성산읍 성산등용로 129-21,성산리 347-9,0.0,21,33.472486,126.932307,21
268,269,0.0,0.447627,0.0,한남시험림,"휴양림,산림욕장",제주 서귀포시 남원읍 서성로651번길 235,한남리 산3,0.0,21,33.336556,126.647308,21


In [2]:
data = recommended_tourist_data[:10]

In [3]:
data

,modelid,rating_score,review_score,price_score,name,type,address1,address2,rating,themenum,latitude,longitude,themenum.1
3,4,0.945485,0.791577,0.513251,용머리해안,"해수욕장,해변",없음,제주 서귀포시 안덕면 사계리 112-3,4.58,21,33.233482,126.314631,21
18,19,0.913472,0.813589,0.564752,만장굴,문화재,제주 제주시 구좌읍 만장굴길 182,월정리 산41-5,4.35,21,33.527589,126.771659,21
25,26,0.930380,0.693470,0.532744,대포해안주상절리대,기념물,제주 서귀포시 이어도로 36-24,중문동 2757-10,4.47,21,33.238700,126.426444,21
31,32,0.930380,0.808440,0.535549,천제연폭포,폭포,제주 서귀포시 천제연로 132 천제연폭포관리소,중문동 2232,4.47,21,33.252366,126.418336,21
42,43,0.935915,0.827989,0.653461,제주김녕미로공원,"식물원,수목원",제주 제주시 구좌읍 만장굴길 122,김녕리 산16,4.51,21,33.536583,126.772390,21
48,49,0.935915,0.706501,0.486749,서귀포자연휴양림,"휴양림,산림욕장",제주 서귀포시 1100로 882,대포동 산1-8,4.51,21,33.310780,126.459777,21
49,50,0.946840,0.740535,0.564752,서귀포 치유의 숲,"휴양림,산림욕장",제주 서귀포시 산록남로 2271,호근동 산4,4.59,21,33.292055,126.529512,21
53,54,0.954909,0.664522,0.513251,거문오름,오름,제주 제주시 조천읍 선교로 569-36,선흘리 478,4.65,21,33.456997,126.714306,21
57,58,0.928988,0.682115,0.486749,붉은오름자연휴양림,"휴양림,산림욕장",제주 서귀포시 표선면 남조로 1487-73,가시리 산158,4.46,21,33.396041,126.669145,21
70,71,0.913472,0.788579,0.632833,동백포레스트,"식물원,수목원",제주 서귀포시 남원읍 생기악로 53-38,신례리 1767,4.35,21,33.300345,126.636216,21


In [4]:
data.to_csv('./user_select_sight.csv',encoding='utf-8',index=False)